In [1]:
import re
from sys import prefix
import pandas as pd
import random
import os
import shutil
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import math


class sudoku:
    def __init__(self, init=False, path='benchmarks', solver='sudoku_acs', execl='charts/base.xlsx', opt_path = '/INST_49x49/100'):
        self.path = path
        self.opt_path = path + opt_path
        self.solver = solver
        self.data_path = 'results/' + solver
        self.records = pd.read_excel(execl, header=[0], index_col = [0])
        if init:
            self.records = self.records.drop(index=self.records.index)
            self.get_info(path)
            
    def get_benchmarks(self, percentage=0):
        file_list = os.listdir(self.opt_path)
        target_path = self.opt_path[:-3] + str(percentage)
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        for file in tqdm(file_list, target_path):
            path = os.path.join(self.opt_path, file)
            with open(path) as f:
                grid = []
                order = f.readline()[:-1]
                is_sat = f.readline()
                square = int(order)*int(order)
                for i in range(square):
                    grid.append(f.readline().split())
                index = 0
                total = (100-percentage)*square*square/100
                while index < total:
                    x = random.randint(0, square-1)
                    y = random.randint(0, square-1)
                    if grid[x][y] != '-1':
                        grid[x][y] = '-1'
                        index += 1
                filename = file.split('_')
                filename[1] = str(percentage)
                filename = '_'.join(filename)
                with open(target_path + '/' + filename, 'w') as out:
                    out.write(order + '\n')
                    out.write(is_sat)
                    for i in range(square):
                        line = ''
                        for j in range(square):
                            line = line + grid[i][j] + '\t'
                        out.write(line[:-1] + '\n')

    def get_empty_benchmarks(self, path='benchmarks/INST_64x64/0'):
        square = int(path.split('/')[1].split('x')[-1])
        for i in range(100):
            filepath = f'{path}/inst{str(square)}x{str(square)}_0_{str(i)}.txt'
            # filepath = path + '/inst64x64_0_' + str(i) + '.txt'
            with open(filepath, 'w') as f:
                f.write(str(math.sqrt(square)) + '\n') 
                f.write('1\n')
                for _ in range(square):
                    line = ''
                    for _ in range(square):
                        line = line + '-1' + '\t'
                    f.write(line[:-1] + '\n')

    def get_part_benchmarks(self):
        order_list = os.listdir(self.path)
        if not os.path.exists('benchmarks_part'):
            os.makedirs('benchmarks_part')
        for order in order_list:
            order_path = os.path.join('benchmarks_part', order)
            if not os.path.exists(order_path):
                os.makedirs(order_path)
            for i in range(0, 105, 5):
                percentage_path = os.path.join(order_path, str(i))
                if not os.path.exists(percentage_path):
                    os.makedirs(percentage_path)
                v1 = percentage_path.replace('_part','')
                v2 = order.split('_')[-1]
                v3 = str(i)
                origin_path = f'{v1}/inst{v2}_{v3}_0.txt'
                shell = f'cp {origin_path} {percentage_path}'
                # print(shell)
                os.system(shell)

    def get_CP_benchmarks(self, path='benchmarks_CP_part'):
        order_list = os.listdir(path)
        for order in order_list:
            order_path = os.path.join(path, order)
            for i in range(0, 105, 5):
                percentage_path = os.path.join(order_path, str(i))
                file_list = os.listdir(percentage_path)
                for file in file_list:
                    file_path = os.path.join(percentage_path, file)
                    target_path = file_path[:-4] + '.dzn'
                    shell = f'mv {file_path} {target_path}'
                    os.system(shell)
                    lines = []
                    with open(target_path) as f:
                        lines = f.readlines()
                    order = int(lines[0][:-1])
                    
                    lines[0] = f'S={str(order)};\n'
                    lines[1] = 'start=[|\n'
                    for i in range(order*order-1):
                        lines[i+2] = lines[i+2].strip('\t\n').replace('\t', ',\t') + '|\n'
                    lines[-1] = lines[-1].strip('\t\n').replace('\t', ',\t') + '|];\n'
                    with open(target_path, 'w') as f:
                        f.writelines(lines)
         
        

    def get_info(self, path):
        folder_list = os.listdir(path)
        for folder in folder_list:
            new_path = os.path.join(path, folder)
            if os.path.isdir(new_path):
                self.get_info(new_path)
            else:
                filename = new_path.split('/')[-1]
                category = filename.split('_')[0]
                percentage = filename.split('_')[1]
                self.records.loc[len(self.records.index)] = [filename, category, percentage]
        
    def save_to(self, filename):
        self.records.to_excel(filename)
        
    def arrange(self):
        # for filename in tqdm(folder_list, desc=folder_path):
        category_list = os.listdir(self.data_path)
        for folder in category_list:
            folder_path = os.path.join(self.data_path, folder)
            file_list = os.listdir(folder_path)
            for file in tqdm(file_list, desc=folder_path):
                path = os.path.join(folder_path, file)
                with open(path) as f:
                    try:
                        if self.solver == 'sudoku_acs':
                            while f.readline():
                                result = f.readline()[:-1]
                                if result == '0':
                                    result = 'sat'
                                elif result == '1':
                                    result = 'timeout'
                                time = f.readline()[:-1]
                                if result == 'timeout':
                                    time = '1000.000000'
                                info = f.readline()
                                # time = info.split(' : ')[-1][:-5]
                                name = info.split(' : ')[0].split('/')[-1]
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_lsc':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                if 'find answer' in info:
                                    result = 'sat'
                                    time = info.split(': ')[-1]
                                    f.readline()
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                    except:
                        print(path)
   

初始时，生成未填充的sudoku，用于求解得到完全填充的sudoku

In [ ]:
# a = sudoku()
# a.get_empty_benchmarks()

初始时，由完全填充的sudoku，删除部分值得到部分填充的benchmarks

In [ ]:
# a = sudoku(opt_path='/INST_81x81/100')
# for i in range(0, 100, 5):
#     a.get_benchmarks(percentage=i)

获取部分benchmarks（每个类型只有一个文件）用于测试

In [27]:
a = sudoku()
a.get_part_benchmarks()
# for i in range(0, 105, 5):
#     print(i)

获取用于CP的benchmarks

In [2]:
a = sudoku()
a.get_CP_benchmarks(path='benchmarks_part_CP')

初始化表格

In [ ]:
a = sudoku(init=True)
a.save_to('charts/base.xlsx')

sudoku_acs 生成表格

In [ ]:
b = sudoku(solver='sudoku_acs')
b.arrange()
b.save_to('charts/sudoku_acs.xlsx')

sudoku_lsc 生成表格

In [ ]:
c = sudoku(solver='sudoku_lsc', execl='charts/sudoku_acs.xlsx')
c.arrange()
c.save_to('charts/sudoku.xlsx')